# IMPORTS

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score



## LEITURA DOS DADOS

In [2]:
df = pd.read_csv('/home/cid34senhas/Desktop/DAATP/Data/5. Data APOS 1_2 sem desvio padrao.csv') 
df_test = pd.read_csv('/home/cid34senhas/Desktop/DAATP/process/Main_Versao/Com CN-MCI/Processamento/DF_Test_1_After_Pre_Processamento.csv')


label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}



# Tratamento

### Funcao Axuliar: Matrix de Confusão

In [3]:
import seaborn as sns
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

## Devisão Origem e Destino

In [4]:
print(df['Transition'].dtype)
print(df['Transition'])


label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}

# Criar o dicionário de reverse mapping
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Aplicar o reverse mapping à coluna Transition
df['Transition'] = df['Transition'].map(reverse_label_mapping)

df['Transition'] = df['Transition'].astype(str)

df[['Origem', 'Destino']] = df['Transition'].str.split('-', expand=True)


int64
0      0
1      0
2      1
3      0
4      2
      ..
290    0
291    0
292    2
293    3
294    0
Name: Transition, Length: 295, dtype: int64


In [5]:
def gerar_transition_previsao(row, stats):
    # Combinações válidas
    combinacoes_validas = {'CN-CN', 'AD-AD', 'MCI-AD', 'MCI-MCI', 'CN-MCI'}

    # Criar a combinação da origem e destino previstos
    transition = f"{row['Origem_Prevista']}-{row['Destino_Prevista']}"

    # Verificar se a combinação é válida
    if transition in combinacoes_validas:
        return transition
    else:
        # Regras para combinações inválidas
        if row['Destino_Prevista'] == 'MCI' and row['Origem_Prevista'] not in {'MCI', 'CN'}:
            stats['alterados'] += 1
            return 'MCI-MCI'
        elif row['Origem_Prevista'] == 'CN' and row['Destino_Prevista'] not in {'MCI', 'CN'}:
            stats['alterados'] += 1
            return 'CN-MCI'
        elif row['Origem_Prevista'] == 'AD' and row['Destino_Prevista'] != 'AD':
            stats['alterados'] += 1
            return 'AD-AD'
        elif row['Origem_Prevista'] == 'MCI' and row['Destino_Prevista'] not in {'MCI', 'AD'}:
            stats['alterados'] += 1
            return 'MCI-MCI'
        else:
            # Contar os casos que não foram corrigidos por falta de regra
            stats['nao_corrigidos'] += 1
            return transition  # Retorna o original, mesmo que inválido


In [6]:
def gerar_origem(df, modelo_origem): 
    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_transtion = df['Transition']
    # Dividir o conjunto de dados em treino e teste para validação

    # Divisão principal para garantir que os índices de teste sejam os mesmos
    X_train_transition, X_test_transition, y_train_transition, y_test_transition = train_test_split(
        X, y_transtion, test_size=0.40, random_state=34, stratify=y_transtion
    )

    # Usar os mesmos índices de treino e teste para y_origem e y_destino
    y_train_origem, y_test_origem = y_origem[X_train_transition.index], y_origem[X_test_transition.index]
    y_train_destino, y_test_destino = y_destino[X_train_transition.index], y_destino[X_test_transition.index]

    X_train_origem, X_test_origem = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    X_train_destino, X_test_destino = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    #


    # ? ORIGEM #########################################################
    # Criar o modelo Random Forest
    rf_origem_model = modelo_origem
    # Avaliar desempenho com cross-validation no treino
    rf_score_origem = cross_val_score(rf_origem_model, X_train_origem, y_train_origem, cv=5, scoring='f1_macro')
    #print("F1 Macro (Cross-Validation):", rf_score_origem.mean())
    # Treinar o modelo no conjunto de treino e prever no conjunto de teste
    rf_origem_model.fit(X_train_origem, y_train_origem)
    rf_origem_pred = rf_origem_model.predict(X_test_origem)
    # Avaliar o desempenho no conjunto de teste
   # print("Relatório de Classificação (Teste):")
   # print(classification_report(y_test_origem, rf_origem_pred))


    report_dict = classification_report(
    y_test_origem, 
    rf_origem_pred,    output_dict=True)


    return report_dict, rf_score_origem.mean(), rf_score_origem.std()

In [7]:
from sklearn.preprocessing import LabelEncoder

def processar_destino_com_label_encoder(df, modelo_destino):
    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_transition = df['Transition']

    # Codificar as classes para y_origem, y_destino e y_transition
    label_encoder_origem = LabelEncoder()
    y_origem_encoded = label_encoder_origem.fit_transform(y_origem)

    label_encoder_destino = LabelEncoder()
    y_destino_encoded = label_encoder_destino.fit_transform(y_destino)

    label_encoder_transition = LabelEncoder()
    y_transition_encoded = label_encoder_transition.fit_transform(y_transition)

    # Dividir o conjunto de dados em treino e teste para validação
    X_train_transition, X_test_transition, y_train_transition, y_test_transition = train_test_split(
        X, y_transition_encoded, test_size=0.40, random_state=34, stratify=y_transition_encoded
    )

    # Usar os mesmos índices de treino e teste para y_origem e y_destino
    y_train_origem, y_test_origem = y_origem_encoded[X_train_transition.index], y_origem_encoded[X_test_transition.index]
    y_train_destino, y_test_destino = y_destino_encoded[X_train_transition.index], y_destino_encoded[X_test_transition.index]

    X_train_origem, X_test_origem = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    X_train_destino, X_test_destino = X.loc[X_train_transition.index], X.loc[X_test_transition.index]

    # Treinar e avaliar o modelo de Origem
    rf_score_destino = cross_val_score(modelo_destino, X_train_destino, y_train_destino, cv=5, scoring='f1_macro')
    #print("F1 Macro (Cross-Validation - Origem):", rf_score_origem.mean())
    #print("Desvio Padrão F1 Macro (Origem):", rf_score_origem.std())

    modelo_destino.fit(X_train_destino, y_train_destino)
    destino_pred = modelo_destino.predict(X_test_destino)
    destino_pred_test = modelo_destino.predict(X_test_transition)

   

    #print("Relatório de Classificação (Teste):")
    
    #print(classification_report(y_test_origem, origem_pred, target_names=label_encoder_origem.classes_ ))
    


    report_dict = classification_report(
    y_test_destino, 
    destino_pred_test, 
    output_dict=True, target_names=label_encoder_origem.classes_)
    return  report_dict , rf_score_destino.mean(),  rf_score_destino.std()

In [8]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import classification_report
from xgboost import XGBClassifier


# Modelos
modelos = {
    "ExtraTrees": ExtraTreesClassifier(
        n_estimators=447, max_features='sqrt', max_depth=15,
        min_samples_split=10, min_samples_leaf=5, bootstrap=False, random_state=34
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=151, max_features='log2', max_depth=13,
        min_samples_split=10, min_samples_leaf=3, bootstrap=True, random_state=34
    ),
    "GradientBoosting": GradientBoostingClassifier(
        learning_rate=0.1, n_estimators=100, random_state=34
    ),
    "KNeighbors": KNeighborsClassifier(
        n_neighbors=5, weights='uniform', algorithm='auto'
    ),
    "SVC": SVC(
        kernel='rbf', random_state=34
    ),
    "XGBOOST" : XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)
}

# Avaliação dos modelos e armazenamento dos resultados
resultados = []
classification_reports = {}

for nome, modelo in modelos.items():
    # Chamar a função de processamento
    report, f1_macro_score, f1_macro_std = processar_destino_com_label_encoder(df, modelo)
    resultados.append({
        "Modelo": nome,
        "F1 Macro (Score Médio)": f1_macro_score,
        "F1 Macro (Desvio Padrão)": f1_macro_std
    })
    
    # Armazenar o classification report para exibição posterior
    classification_reports[nome] = report

# Converter resultados em tabela
tabela_resultados = pd.DataFrame(resultados)

# Exibir tabela formatada
print("\nResultados Resumidos:")
print(tabulate(tabela_resultados, headers="keys", tablefmt="grid"))

# Criar uma lista para armazenar os resultados de F1 macro para os três parâmetros
f1_scores = []

# Exibir classification reports de todos os modelos
print("\nClassification Reports:")
for nome, report in classification_reports.items():
    print(f"\n{nome} - Classification Report")
    df_report = pd.DataFrame(report).transpose()  # Mostrar o report como DataFrame para facilitar leitura
    print(df_report)

    # Extrair os valores de F1 macro para AD, CN e MCI
    f1_ad = report.get('AD', {}).get('f1-score', 0)
    f1_cn = report.get('CN', {}).get('f1-score', 0)
    f1_mci = report.get('MCI', {}).get('f1-score', 0)

    # Armazenar os valores e o nome do modelo
    f1_scores.append((nome, f1_ad, f1_cn, f1_mci))

# Ordenar os modelos com base no F1 macro (média dos três parâmetros)
f1_scores_sorted = sorted(f1_scores, key=lambda x: (x[1] + x[2] + x[3]) / 3, reverse=True)

# Exibir os três melhores modelos com suas respectivas pontuações F1 macro
print("\nTop 3 Models based on F1 Macro Score for AD, CN, and MCI:")
for i, (nome, f1_ad, f1_cn, f1_mci) in enumerate(f1_scores_sorted[:3], 1):
    print(f"{i}. {nome} - F1 (AD): {f1_ad:.4f}, F1 (CN): {f1_cn:.4f}, F1 (MCI): {f1_mci:.4f}")


/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f


Resultados Resumidos:
+----+------------------+--------------------------+----------------------------+
|    | Modelo           |   F1 Macro (Score Médio) |   F1 Macro (Desvio Padrão) |
+====+==================+==========================+============================+
|  0 | ExtraTrees       |                 0.482943 |                  0.045026  |
+----+------------------+--------------------------+----------------------------+
|  1 | RandomForest     |                 0.465133 |                  0.0527457 |
+----+------------------+--------------------------+----------------------------+
|  2 | GradientBoosting |                 0.453584 |                  0.0768293 |
+----+------------------+--------------------------+----------------------------+
|  3 | KNeighbors       |                 0.394377 |                  0.0725168 |
+----+------------------+--------------------------+----------------------------+
|  4 | SVC              |                 0.38086  |                  0.037

In [9]:

# Criar instâncias dos modelos
xgbost_origem = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)

random_forest_origem = RandomForestClassifier(n_estimators=800,random_state=34)


report_xgboost, f1_macro_score_xgboost, f1_macro_std_xgboost = processar_destino_com_label_encoder(df,xgbost_origem)
report_random, f1_macro_score_rf, f1_macro_std_rf = gerar_origem(df,random_forest_origem)

print("Xgboost:")
print(report_xgboost)
print("F1_macro_Score:")
print(f1_macro_score_xgboost)
print("F1_macro_Desvio_padrao:")
print(f1_macro_std_xgboost)



Xgboost:
{'AD': {'precision': 0.711864406779661, 'recall': 0.8235294117647058, 'f1-score': 0.7636363636363637, 'support': 51.0}, 'CN': {'precision': 0.62, 'recall': 0.7948717948717948, 'f1-score': 0.6966292134831461, 'support': 39.0}, 'MCI': {'precision': 0.5555555555555556, 'recall': 0.17857142857142858, 'f1-score': 0.2702702702702703, 'support': 28.0}, 'accuracy': 0.6610169491525424, 'macro avg': {'precision': 0.6291399874450722, 'recall': 0.5989908784026431, 'f1-score': 0.57684528246326, 'support': 118.0}, 'weighted avg': {'precision': 0.6444122059433751, 'recall': 0.6610169491525424, 'f1-score': 0.6244200121937695, 'support': 118.0}}
F1_macro_Score:
0.44178512534132686
F1_macro_Desvio_padrao:
0.0512870761142803
